In [1]:
import py_entitymatching as em
import os, sys
import pandas as pd

In [2]:
#datasets directory
#datasets_dir = "/home/zeus/data_science/datasets/datasets"
datasets_dir = "/Users/nehamittal/Downloads/datasets"
#input tables
path_songs = datasets_dir + os.sep +'songs.csv'
path_tracks= datasets_dir+ os.sep + "tracks.csv"

In [3]:
songs = em.read_csv_metadata(path_songs, key='id',low_memory=False)
tracks = em.read_csv_metadata(path_tracks, key='id',low_memory=False)
#schema of both songs and tracks- id,title,artist,year

No handlers could be found for logger "py_entitymatching.io.parsers"


In [4]:
# We have labeled the data and stored it in 
# sampled.csv
path_labeled_data = datasets_dir + os.sep + 'sampled.csv'
G = em.read_csv_metadata(path_labeled_data,key='_id',
                         ltable=songs, rtable=tracks,
                         fk_ltable='ltable_id', fk_rtable='rtable_id'
                         )
G['key'] = range(0, len(G))
em.set_key(G, 'key')

True

In [5]:
G.head()

In [6]:
# Split S into I an J
#em.get_key(G)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']
#I.to_csv(datasets_dir + "/" +"SetI.csv")
#J.to_csv(datasets_dir + "/" +"SetJ.csv")

In [7]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name="NaiveBayes")

In [8]:
# Generate a set of features
F = em.get_features_for_matching(songs, tracks)
F.feature_name

0                             id_id_exm
1                             id_id_anm
2                        id_id_lev_dist
3                         id_id_lev_sim
4           title_title_jac_qgm_3_qgm_3
5       title_title_cos_dlm_dc0_dlm_dc0
6       title_title_jac_dlm_dc0_dlm_dc0
7                       title_title_mel
8                  title_title_lev_dist
9                   title_title_lev_sim
10                      title_title_nmw
11                       title_title_sw
12        artist_artist_jac_qgm_3_qgm_3
13    artist_artist_cos_dlm_dc0_dlm_dc0
14    artist_artist_jac_dlm_dc0_dlm_dc0
15                    artist_artist_mel
16               artist_artist_lev_dist
17                artist_artist_lev_sim
18                    artist_artist_nmw
19                     artist_artist_sw
20                        year_year_exm
21                        year_year_anm
22                   year_year_lev_dist
23                    year_year_lev_sim
Name: feature_name, dtype: object

In [9]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='gold_label',
                            show_progress=False)  

In [10]:
# Display first few rows
H.head()

In [11]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(H))

True

In [12]:
# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'],
                strategy='mean')

In [13]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg,nb], table=H, 
        exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'],
        k=5,
        target_attr='gold_label', metric='f1', random_state=0)
result['cv_stats'].to_csv(datasets_dir + "/" +"F1.csv")

In [14]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg,nb], table=H, 
        exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'],
        k=5,
        target_attr='gold_label', metric='precision', random_state=0)
result['cv_stats'].to_csv(datasets_dir + "/" +"precison.csv")

In [15]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg,nb], table=H, 
        exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'],
        k=5,
        target_attr='gold_label', metric='recall', random_state=0)
result['cv_stats'].to_csv(datasets_dir + "/" +"recall.csv")

In [16]:
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.7, random_state=0)
P = PQ['train']
Q = PQ['test']

In [17]:
# Debug RF matcher using GUI
em.vis_debug_rf(rf, P, Q, 
        exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'],
        target_attr='gold_label')

In [18]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

dt.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

svm.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

ln.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

lg.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

nb.fit(table=H, 
       exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
       target_attr='gold_label')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='gold_label', show_progress=False)



In [19]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 92.86% (52/56)
Recall : 92.86% (52/56)
F1 : 92.86%
False positives : 4 (out of 56 positive predictions)
False negatives : 4 (out of 51 negative predictions)


In [20]:
# Predict on L 
predictions = dt.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 87.93% (51/58)
Recall : 91.07% (51/56)
F1 : 89.47%
False positives : 7 (out of 58 positive predictions)
False negatives : 5 (out of 49 negative predictions)


In [21]:
# Predict on L 
predictions = svm.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions

eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 94.64% (53/56)
Recall : 94.64% (53/56)
F1 : 94.64%
False positives : 3 (out of 56 positive predictions)
False negatives : 3 (out of 51 negative predictions)


In [22]:
# Predict on L 
predictions = ln.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 98.11% (52/53)
Recall : 92.86% (52/56)
F1 : 95.41%
False positives : 1 (out of 53 positive predictions)
False negatives : 4 (out of 54 negative predictions)


In [23]:
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 94.55% (52/55)
Recall : 92.86% (52/56)
F1 : 93.69%
False positives : 3 (out of 55 positive predictions)
False negatives : 4 (out of 52 negative predictions)


In [24]:
# Predict on L 
predictions = nb.predict(table=L, exclude_attrs=['key', 'ltable_id', 'rtable_id', 'gold_label'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 96.15% (50/52)
Recall : 89.29% (50/56)
F1 : 92.59%
False positives : 2 (out of 52 positive predictions)
False negatives : 6 (out of 55 negative predictions)
